# Load the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
#from scipy.optimize import minimize
from scipy import integrate
#from iminuit import Minuit
#from iminuit.cost import UnbinnedNLL
from landaupy import landau
from scipy.optimize import curve_fit

In [ ]:
def landau_distribution(xdata, mu,sigma, A):
    landau = lambda t, mu, sigma, xdata, A : A*np.exp(-t)*np.cos(t*(xdata -mu)/sigma + 2*t/np.pi *np.log(t/sigma) ) / (sigma *  np.pi)
    integral, error = integrate.quad(landau, 0, np.inf, args=(mu,sigma,xdata, A))
    return integral

def landau_distribution_array(xdata,mu, sigma, A):
    return np.array([landau_distribution(x, mu,sigma, A) for x in xdata])

def landau_fit(xdata, location, scale, A):
    mu = location
    sigma = scale
    return A*landau.pdf(xdata, mu, sigma)

def gaussian(x, a, mu, sigma):
    return a * np.exp(-(x - mu)**2 / (2 * sigma**2))

def load_runlist_to_numpy(runlist_path):
    with open(runlist_path, 'r') as file:
        runlist = file.readlines()
    runlist = [run.strip() for run in runlist]
    return np.array(runlist)
    
def load_runlist_to_numpy(runlist_path):
    with open(runlist_path, 'r') as file:
        runlist = file.readlines()
    runlist = [run.strip() for run in runlist]
    return np.array(runlist)

def plot_waveform(wave, baseline, st, ed, lp):
    import matplotlib.pyplot as plt
    import numpy as np
    plt.figure()
    plt.step(np.arange(len(wave)), wave, where='mid')
    plt.axhline(y=baseline, color='b', linestyle='--', label='Baseline')  # 基线
    plt.axhline(y=baseline -20, color='r', linestyle='--', label='Threshold')  # 阈值
    plt.scatter(st, wave[st], color='r', marker='o', label='start')  # 起始点
    plt.scatter(ed, wave[ed], color='g', marker='o', label='end')  # 结束点
    plt.scatter(lp, wave[lp], color='b', marker='o', label='minpoint')  # 最低点
    plt.xlabel('Time [4 ns]')
    plt.ylabel('ADC Count')
    plt.legend()
    plt.xlim(80,150)
    plt.show()

# build an list or array to store run information

In [ ]:
run_info =[]
runlist = 'runlist/LED_20240821_20240826_h5py_runlist_test'
runs = load_runlist_to_numpy(runlist)
print(runs)
len(runs)

In [ ]:
for file in runs:
    file_date = file.split('LED_')[0].split('readout_')[2]
    led_config = file.split('LED_')[1].split('_400')[0]
    file_tag = file_date + led_config
    #print(file_tag)
    df = pd.read_hdf(file, key='winfo')
    area_ch0 = df.Area[df.Ch==0 ].astype(np.float64).to_numpy()
    area_ch1 = df.Area_fixrange[df.Ch==1 ].astype(np.float64).to_numpy()
    #df.head()
    ratio = area_ch0/area_ch1
    mean_ch0 = np.mean(area_ch0)
    std_ch0 = np.std(area_ch0)
    nbins_ch0 = round(np.max(area_ch0)-np.min(area_ch0))
    left_edge_ch0 = np.min(area_ch0)
    right_edge_ch0 = np.max(area_ch0)
    amplitude_ch0 = len(area_ch0)
    mean_ch1 = np.mean(area_ch1)
    std_ch1 = np.std(area_ch1)
    nbins_ch1 = round(np.max(area_ch1)-np.min(area_ch1))
    amplitude_ch1 = len(area_ch1)
    left_edge_ch1 = np.min(area_ch1)
    right_edge_ch1 = np.max(area_ch1)

    if nbins_ch0 > 300:
        nbins_ch0 = 200
        left_edge_ch0 = mean_ch0 -100
        right_edge_ch0 = mean_ch0 + 100
    if nbins_ch1 > 200:
        nbins_ch1 = 200
        left_edge_ch1 = mean_ch1 -100
        right_edge_ch1 = mean_ch1 + 100
    
    hist, bins_edges = np.histogram(area_ch0, bins= nbins_ch0, range=(left_edge_ch0, right_edge_ch0))
    bins = (bins_edges[:-1] + bins_edges[1:])/2
    popt, pcov = curve_fit(gaussian, bins, hist, p0=[amplitude_ch0, mean_ch0,  std_ch0])
    x_fit = np.linspace(np.min(bins), np.max(bins), 1000)
    y_fit = gaussian(x_fit, *popt)
    plt.plot(x_fit, y_fit, label=r'$\mu$={:.2f}, $\sigma$={:.2f}'.format(popt[1], popt[2]))
    #plt.hist(area_ch0, bins=nbins_ch0, range=(left_edge_ch0, right_edge_ch0),  color='black', density=False, alpha=0.5, label='Ch0 Area')
    plt.hist(area_ch0, bins=nbins_ch0, range=(left_edge_ch0, right_edge_ch0),  color='black', density=False, alpha=0.5, label='Ch0 Area')
    plt.xlabel('Ch0 Area[PE]')
    plt.ylabel('Entries')
    plt.title(r'LED {} Ch0 Area'.format(file_tag))
    plt.legend()
    plt.savefig(r'./figs/LED_{}_Ch0_Area_fit_{:.2f}_PE_sigma_{:.2f}.png'.format(file_tag,popt[1], popt[2]),dpi=300)
    plt.show()
    plt.cla()
    print(r'Ch0 Area Fit: mu= {:.2f}, sigma ={:.2f}'.format(popt[1], popt[2]))
    Ch0_area_pe_mu = popt[1]
    Ch0_area_pe_sigma = popt[2]
    
    hist_, bins__edges_ = np.histogram(area_ch1, bins= nbins_ch1, range=(left_edge_ch1, right_edge_ch1))
    bins_ = (bins__edges_[:-1] + bins__edges_[1:])/2
    popt, pcov = curve_fit(gaussian, bins_, hist_, p0=[1.E3, mean_ch1,  std_ch1])
    x_fit = np.linspace(np.min(bins_), np.max(bins_), 1000)
    y_fit = gaussian(x_fit, *popt)
    plt.plot(x_fit, y_fit, label=r'$\mu$={:.2f}, $\sigma$={:.2f}'.format(popt[1], popt[2]))
    #plt.hist(area_ch0, bins=nbins_ch0, range=(left_edge_ch0, right_edge_ch0),  color='black', density=False, alpha=0.5, label='Ch0 Area')
    plt.hist(area_ch1, bins=nbins_ch1, range=(left_edge_ch1, right_edge_ch1),  color='black', density=False, alpha=0.5, label='Ch0 Area')
    plt.xlabel('Ch1 Area[PE]')
    plt.ylabel('Entries')
    plt.title(r'LED {} Ch1 Area'.format(file_tag))
    plt.legend()
    plt.savefig(r'./figs/LED_{}_Ch1_Area_fit_{:.2f}_PE_sigma_{:.2f}.png'.format(file_tag,popt[1], popt[2]),dpi=300)
    plt.show()
    plt.cla()
    print(r'Ch1 Area Fit: mu= {:.2f}, sigma ={:.2f}'.format(popt[1], popt[2]))
    Ch1_area_pe_mu = popt[1]
    Ch1_area_pe_sigma = popt[2]
    
    plt.hist2d(area_ch0, area_ch1, bins=[
        np.linspace(left_edge_ch0, right_edge_ch0, nbins_ch0),
        np.linspace(left_edge_ch1, right_edge_ch1-mean_ch1, nbins_ch1)
    ] , norm=LogNorm(), cmap='jet', label='Ch1_vs_Ch0_area')
    plt.colorbar()
    plt.xlabel('Ch0 Area[ADC]')
    plt.ylabel('Ch1 Area[ADC]')
    plt.legend()
    plt.savefig(r'./figs/LED_{}_Ch0_{:.2f}_PE_Ch1_{:.2f}_PE.png'.format(file_tag, Ch0_area_pe_mu, Ch1_area_pe_mu),dpi=300)
    plt.show()
    plt.cla()
    

    
    plt.hist2d(area_ch0, ratio, bins=[
        np.linspace(left_edge_ch0,right_edge_ch0, nbins_ch0),
        np.linspace(0, 250, 200)
    ] , norm=LogNorm(), cmap='jet', label='Ch0_Ch1_ratio_vs_Ch0_area')
    plt.colorbar()
    #plt.yscale('log')
    plt.xlabel('Ch0 Area[PE]')
    plt.ylabel('Ch0/Ch1 Ratio')
    plt.title(r'LED {} Ch0 Area vs Ch0/Ch1 Ratio'.format(file_tag))
    plt.legend()
    plt.savefig(r'./res/LED_{}_Ch0_Area_vs_Ch0_Ch1_Ratio.png'.format(file_tag),dpi=300)
    plt.show()
    plt.cla()
    
    hist, bins_edges =np.histogram(ratio, bins=200, range=(0, 250))
    bins = (bins_edges[:-1] + bins_edges[1:])/2
    popt, pcov = curve_fit(landau_fit, bins, hist , p0=[30, 10, 1.4e4])
    
    xfit = np.linspace(0, 250, 1000)
    y_fit = landau_fit(x_fit, popt[0], popt[1],popt[2])
    #fig, ax = plt.subplots()
    plt.hist(ratio, bins=200, range=(0, 250), density=False, alpha=0.5, label='Ratio')
    plt.plot(x_fit, y_fit, 'r-',color='black',  label=r'$\mu$={:.2f}, $\sigma$={:.2f}, A={:.2f}'.format(popt[0],popt[1],popt[2]))
    plt.xlabel('Ch0/CH1 Ratio')
    plt.ylabel('PDF')
    plt.title('Landau Distribution')
    plt.legend()
    plt.savefig(r'./figs/LED_{}_Ch0_CH1_Ratio_fit_mu_{:.2f}_sigma_{:.2f}.png'.format(file_tag,popt[0],popt[1] ),dpi=300)
    plt.show()
    print(r'Ratio Fitting Result: mu= {:.2f}, sigma= {:.2f}'.format(popt[0],popt[1]))
    Ratio_fit_mu = popt[0]
    Ratio_fit_sigma = popt[1]
    
    run_info.append({
        'Ch0_Area_mu': Ch0_area_pe_mu,
        'Ch0_Area_sigma': Ch0_area_pe_sigma,
        'Ratio_fit_mu': Ratio_fit_mu,
        'Ratio_fit_sigma': Ratio_fit_sigma,
        'file_tag': file_tag    
    })

In [ ]:
import process_data
path_save = 'outnpy/LED_20240821_20240826_runinfo.h5py'
df = pd.DataFrame(run_info)
process_data.write_to_hdf5(df, path_save)

In [ ]:
df

# load data 

In [ ]:
############################################ run file 20240821 h5py#####################
file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240821_LED_1.7V_11ns_400_ratio_run0.h5py'
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240821_LED_1.7V_12ns_400_ratio_run0.h5py'
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240821_LED_1.7V_13ns_400_ratio_run0.h5py'
#file  = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240821_LED_1.7V_14ns_400_ratio_run0.h5py'
#file  = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240821_LED_1.7V_15ns_400_ratio_run0.h5py'
############################################ run file 20240826 h5py###################
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.175V_50ns_400_ratio_run0.h5py'
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.175V_47ns_400_ratio_run0.h5py'
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.175V_48ns_400_ratio_run0.h5py'
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.175V_49ns_400_ratio_run0.h5py'
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.175V_46ns_400_ratio_run0.h5py'
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.130V_65ns_400_ratio_run0.h5py'
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.13V_60ns_400_ratio_run0.h5py'
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.13V_61ns_400_ratio_run0.h5py'
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.13V_63ns_400_ratio_run0.h5py'
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.13V_63ns_400_ratio_run0_test.h5df'
#file = 'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.13V_63ns_400_ratio_run0_debug_area.h5df'


In [ ]:
file_date = file.split('LED_')[0].split('readout_')[2]
led_config = file.split('LED_')[1].split('_400')[0]
file_tag = file_date + led_config
file_tag

In [ ]:
df = pd.read_hdf(file, key='winfo')
df.head()

In [ ]:
#n =301862
#wave = df_ch1_diff.Wave[:][n]
#baseline = df_ch1_diff.Baseline[:][n]
#st = df_ch1_diff.St[:][n]
#ed = df_ch1_diff.Ed[:][n]
#lp = df_ch1_diff.Minp[:][n]


In [ ]:
#plot_waveform(wave, baseline, st, ed, lp)

In [ ]:
area_ch0 = df.Area[df.Ch==0 ].astype(np.float64).to_numpy()
area_ch1 = df.Area_fixrange[df.Ch==1 ].astype(np.float64).to_numpy()

In [ ]:
area_ch0, area_ch1

# define the range and number of bins to fit histogram of area distribution

In [ ]:
ratio = area_ch0/area_ch1
mean_ch0 = np.mean(area_ch0)
std_ch0 = np.std(area_ch0)
nbins_ch0 = round(np.max(area_ch0)-np.min(area_ch0))
left_edge_ch0 = np.min(area_ch0)
right_edge_ch0 = np.max(area_ch0)
amplitude_ch0 = len(area_ch0)
mean_ch1 = np.mean(area_ch1)
std_ch1 = np.std(area_ch1)
nbins_ch1 = round(np.max(area_ch1)-np.min(area_ch1))
left_edge_ch1 = np.min(area_ch1)
right_edge_ch1 = np.max(area_ch1)
amplitude_ch1 = len(area_ch1)

In [ ]:
mean_ch0, std_ch0 ,left_edge_ch0 , right_edge_ch0, nbins_ch0 , amplitude_ch0

In [ ]:

if nbins_ch0 > 300:
    nbins_ch0 = 200
    left_edge_ch0 = mean_ch0 -100
    right_edge_ch0 = mean_ch0 + 100

In [ ]:
#hist, bins_edges = np.histogram(area_ch0, bins= nbins_ch0, range=(left_edge_ch0, right_edge_ch0))
hist, bins_edges = np.histogram(area_ch0, bins= nbins_ch0, range=(left_edge_ch0, right_edge_ch0))
bins = (bins_edges[:-1] + bins_edges[1:])/2
popt, pcov = curve_fit(gaussian, bins, hist, p0=[amplitude_ch0, mean_ch0,  std_ch0])
x_fit = np.linspace(np.min(bins), np.max(bins), 1000)
y_fit = gaussian(x_fit, *popt)
plt.plot(x_fit, y_fit, label=r'$\mu$={:.2f}, $\sigma$={:.2f}'.format(popt[1], popt[2]))
#plt.hist(area_ch0, bins=nbins_ch0, range=(left_edge_ch0, right_edge_ch0),  color='black', density=False, alpha=0.5, label='Ch0 Area')
plt.hist(area_ch0, bins=nbins_ch0, range=(left_edge_ch0, right_edge_ch0),  color='black', density=False, alpha=0.5, label='Ch0 Area')
plt.xlabel('Ch0 Area[PE]')
plt.ylabel('Entries')
plt.title(r'LED {} Ch0 Area'.format(file_tag))
plt.legend()
plt.savefig(r'./res/LED_{}_Ch0_Area_fit_{:.2f}_PE_sigma_{:.2f}.png'.format(file_tag,popt[1], popt[2]),dpi=300)
plt.show()
print(r'Ch0 Area Fit: mu= {:.2f}, sigma ={:.2f}'.format(popt[1], popt[2]))
Ch0_area_pe_mu = popt[1]
Ch0_area_pe_sigma = popt[2]

In [ ]:
mean_ch1, std_ch1 ,left_edge_ch1 , right_edge_ch1, nbins_ch1, amplitude_ch1

In [ ]:
if nbins_ch1 > 300:
    nbins_ch1= 200
    left_edge_ch1 = mean_ch1 -100
    right_edge_ch1= mean_ch1 + 100

In [ ]:
#hist, bins_edges = np.histogram(area_ch0, bins= nbins_ch0, range=(left_edge_ch0, right_edge_ch0))
hist_, bins__edges_ = np.histogram(area_ch1, bins= nbins_ch1, range=(left_edge_ch1, right_edge_ch1))
bins_ = (bins__edges_[:-1] + bins__edges_[1:])/2
popt, pcov = curve_fit(gaussian, bins_, hist_, p0=[1.E4, mean_ch1,  20])
x_fit = np.linspace(np.min(bins_), np.max(bins_), 1000)
y_fit = gaussian(x_fit, *popt)
plt.plot(x_fit, y_fit, label=r'$\mu$={:.2f}, $\sigma$={:.2f}'.format(popt[1], popt[2]))
#plt.hist(area_ch0, bins=nbins_ch0, range=(left_edge_ch0, right_edge_ch0),  color='black', density=False, alpha=0.5, label='Ch0 Area')
plt.hist(area_ch1, bins=nbins_ch1, range=(left_edge_ch1, right_edge_ch1),  color='black', density=False, alpha=0.5, label='Ch0 Area')
plt.xlabel('Ch1 Area[PE]')
plt.ylabel('Entries')
plt.title(r'LED {} Ch1 Area'.format(file_tag))
plt.legend()
plt.savefig(r'./res/LED_{}_Ch0_Area_fit_{:.2f}_PE_sigma_{:.2f}.png'.format(file_tag,popt[1], popt[2]),dpi=300)
plt.show()
print(r'Ch1 Area Fit: mu= {:.2f}, sigma ={:.2f}'.format(popt[1], popt[2]))

In [ ]:
plt.hist2d(area_ch0, area_ch1, bins=[
    np.linspace(left_edge_ch0, right_edge_ch0, nbins_ch0),
    np.linspace(left_edge_ch1, right_edge_ch1-mean_ch1, nbins_ch1)
] , norm=LogNorm(), cmap='jet')
plt.colorbar()
plt.xlabel('Ch0 Area[ADC]')
plt.ylabel('Ch1 Area[ADC]')
plt.show()

In [ ]:
if Ch0_area_pe_mu < 25:
        ratio_right_edge = 30
        ratio_mean = 20
if Ch0_area_pe_mu > 25 and Ch0_area_pe_mu < 50:
        ratio_right_edge = 150
        ratio_mean = 30
if Ch0_area_pe_mu > 50:
        ratio_right_edge = 200
        ratio_mean  =30
if Ch0_area_pe_mu > 300:
        ratio_right_edge = 200
        ratio_mean  =40
if Ch0_area_pe_mu > 500:
        ratio_right_edge = 250
        ratio_mean  =50
if mean_ch1 < 10:
        nbins_ch1 = 5

In [ ]:
plt.hist2d(area_ch0, ratio, bins=[
    np.linspace(left_edge_ch0,right_edge_ch0, nbins_ch0),
    np.linspace(0, ratio_right_edge,100)
] , norm=LogNorm(), cmap='jet')
plt.colorbar()
#plt.yscale('log')
plt.xlabel('Ch0 Area[PE]')
plt.ylabel('Ch0/Ch1 Ratio')
plt.title(r'LED {} Ch0 Area vs Ch0/Ch1 Ratio'.format(file_tag))
plt.savefig(r'./res/LED_{}_Ch0_Area_vs_Ch0_Ch1_Ratio.png'.format(file_tag),dpi=300)
plt.show()

# Try to fit ratio dsitrubution of two datasets using Landau distribution

In [ ]:
hist, bins_edges =np.histogram(ratio, bins=200, range=(0, 250))
bins = (bins_edges[:-1] + bins_edges[1:])/2
popt, pcov = curve_fit(landau_fit, bins, hist , p0=[ratio_mean, 10, 5e4])

In [ ]:
popt[0], popt[1], popt[2]

In [ ]:
x_fit = np.linspace(np.min(bins), np.max(bins), 1000)
y_fit = landau_fit(x_fit, popt[0], popt[1],popt[2])
fig, ax = plt.subplots()
ax.hist(ratio, bins=200, range=(0, 250), density=False, alpha=0.5, label='Ratio')
ax.plot(x_fit, y_fit, 'r-',color='black',  label=r'$\mu$={:.2f}, $\sigma$={:.2f}, A={:.2f}'.format(popt[0],popt[1],popt[2]))
ax.set_xlabel('Ch0/CH1 Ratio')
ax.set_ylabel('PDF')
ax.set_title('Landau Distribution')
ax.legend()
plt.savefig(r'./res/LED_{}_Ch0_CH1_Ratio_fit_mu_{:.2f}_sigma_{:.2f}.png'.format(file_tag,popt[0],popt[1] ),dpi=300)
plt.show()
print(r'Ratio Fitting Result: mu= {:.2f}, sigma= {:.2f}'.format(popt[0],popt[1]))
Ratio_fit_mu = popt[0]
Ratio_fit_sigma = popt[1]

In [5]:
hist, bins_edges =np.histogram(ratio, bins=200, range=(0, 250))
bins = (bins_edges[:-1] + bins_edges[1:])/2
popt, pcov = curve_fit(landau_fit, bins, hist , p0=[30, 10, 1.4e4])
   
xfit = np.linspace(0, 250, 1000)
y_fit = landau_fit(x_fit, popt[0], popt[1],popt[2])
plt.hist(ratio, bins=200, range=(0, 250), density=False, alpha=0.5, label='Ratio')
plt.plot(x_fit, y_fit, 'r-',color='black',  label=r'$\mu$={:.2f}, $\sigma$={:.2f}, A={:.2f}'.format(popt[0],popt[1],popt[2]))
    plt.xlabel('Ch0/CH1 Ratio')
    plt.ylabel('PDF')
    plt.title('Landau Distribution')
    plt.legend()
    plt.savefig(r'./figs/LED_{}_Ch0_CH1_Ratio_fit_mu_{:.2f}_sigma_{:.2f}.png'.format(file_tag,popt[0],popt[1] ),dpi=300)
    plt.show()
    print(r'Ratio Fitting Result: mu= {:.2f}, sigma= {:.2f}'.format(popt[0],popt[1]))
    Ratio_fit_mu = popt[0]
    Ratio_fit_sigma = popt[1]

IndentationError: unexpected indent (446765807.py, line 9)

# store the run info to DateFrame

In [ ]:
Ch0_area_pe_mu, Ch0_area_pe_sigma, Ratio_fit_mu, Ratio_fit_sigma, file_tag
run_info.append({
    'Ch0_Area_mu': Ch0_area_pe_mu,
    'Ch0_Area_sigma': Ch0_area_pe_sigma,
    'Ratio_fit_mu': Ratio_fit_mu,
    'Ratio_fit_sigma': Ratio_fit_sigma,
    'file_tag': file_tag    
})

In [ ]:
run_info[ -3:]

In [ ]:
import process_data
path_save = 'outnpy/LED_20240821_20240826_run_info.h5py'
df = pd.DataFrame(run_info)
process_data.write_to_hdf5(df, path_save)

# Plot Ch0 area with PE and Ch0/Ch1 ratio 

In [ ]:
LED_config_1p7V   = np.array(['1.7V_11ns', '1.7V_12ns', '1.7V_13ns', '1.7V_14ns', '1.7V_15ns', '1.7V_15.5ns', '1.7V_16ns', '1.7V_16.6ns', '1.7V_17ns', '1.7V_15ns','1.7V_14.5ns', '1.7V_14ns', '1.7V_13.5ns', 
                              '1.7V_13ns', '1.7V_17.5ns', '1.175V_50ns','1.175V_47ns', '1.175V_48ns', '1.175V_49ns','1.175V_46ns','1.175V_65ns', '1.175V_60ns','1.175V_61ns','1.175V_61ns'  ])
Ch0_area_pe       = np.array([8.99, 30.76, 76.65, 154.51, 268.25, 161.94, 217.5, 283.39, 359.40, 115.72, 79.24, 51.17, 30.86, 17.12, 431.44, 639.75, 465.27, 518.33, 578.45, 415.39, 936.07, 670.12, 716.79, 815.88])
Ch0_area_pe_err   = np.array([3.27, 5.98,  9.38,  13.20,  17.05,  13.42,  15.32, 17.32,  19.30,  11.44,  9.47,  7.62,  5.91,  4.43,  16.50,  26.40,  22.66,  23.76,  25.10,  21.40,  32.06,  27.33,  28.29,  30.00])
Ch0_Ch1_ratio     = np.array([6.90, 19.60, 29.79, 34.93,  37.900, 38.15,  40.03, 41.57,  42.40,  35.67,  32.30, 27.48, 20.51, 12.84, 41.83,  50.88,  48.82,  49.59,  50.21,  48.11,  65.10,  59.72,  60.82,  62.91])
Ch0_Ch1_ratio_err = np.array([2.53, 6.91,  10.26, 9.70,   6.82,   10.77,  10.15, 9.47,   8.69,   11.11,  11.26, 9.95,  7.13,  4.26,  7.76,   10.44,  10.91,  10.72,  10.48,  11.11,  14.66,  14.03,  14.11,  14.39])

fig, ax = plt.subplots(figsize=(8,6))
ax.errorbar(Ch0_area_pe, Ch0_Ch1_ratio, xerr=Ch0_area_pe_err, yerr=Ch0_Ch1_ratio_err, fmt='o',  capsize=5, elinewidth=2, capthick=2, label='Ratio')
ax.set_xlabel('Ch0 Area [PE]')
ax.set_ylabel('Ch0/Ch1 Ratio')
ax.set_title('Ch0/Ch1 Ratio vs Ch0 Area')
ax.legend()
#plt.savefig(r'./res/Ch0_CH1_Ratio_vs_Ch0_Area_with_LED_{}.png'.format(v_value ),dpi=300)
plt.show()

In [ ]:
len(LED_config_1p7V), len(Ch0_area_pe), len(Ch0_area_pe_err), len(Ch0_Ch1_ratio), len(Ch0_Ch1_ratio_err)